In [ ]:
'''
Imports
'''

# General
import json
import numpy as np
from collections import Counter
import pandas as pd

try:
  import wandb
except:
  ! pip install wandb
  import wandb

try:
  from datasets import Dataset
except:
  ! pip install datasets
  from datasets import Dataset

# Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score as sk_precision_score
from sklearn.metrics import recall_score as sk_recall_score
from sklearn.metrics import f1_score as sk_f1_score

try:
  from seqeval.metrics import classification_report as seqeval_classification_report
  from seqeval.metrics import f1_score, precision_score, recall_score
except:
  ! pip install seqeval
  from seqeval.metrics import classification_report as seqeval_classification_report
  from seqeval.metrics import f1_score, precision_score, recall_score

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# Transformers
try:
    from transformers import (
        DataCollatorForTokenClassification,
        DistilBertConfig,
        DistilBertForTokenClassification,
        DistilBertTokenizerFast,
        Trainer,
        TrainingArguments,
    )
except:
    ! pip install transformers
    from transformers import (
        DataCollatorForTokenClassification,
        DistilBertConfig,
        DistilBertForTokenClassification,
        DistilBertTokenizerFast,
        Trainer,
        TrainingArguments,
    )

! pip install accelerate

In [ ]:
import os
#os.environ["WANDB_MODE"] = "offline"
from google.colab import drive
if "COLAB_GPU" in os.environ:
        drive.mount('/content/drive')

output_dir = "/content/drive/MyDrive/distilbert_outputs"
os.makedirs(output_dir, exist_ok=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
"""
    Preprocess a list of email chains for token classification with BIO tagging.

    For each email chain, this function:
    - Concatenates all email bodies into a single string.
    - Extracts entity values from the tenant profile.
    - Tokenizes the email text.
    - Labels each token using the BIO format based on entity spans.
    - Converts labels to IDs using the provided label2id mapping.
    - Masks padding token positions with -100 for loss computation.

    Args:
        data (list): List of email chains, each as a dictionary with an email_chain and tenant_profile.
        tokenizer (PreTrainedTokenizer): Hugging Face tokenizer.
        label2id (dict): Mapping from BIO label strings to integer IDs.
        max_length (int, optional): Maximum sequence length for tokenization. Defaults to 512.

    Returns:
        A tuple of two lists:
            - List of tokenized input dictionaries (with input_ids, attention_mask, etc.)
            - List of corresponding label ID sequences
"""

def preprocess_for_token_classification(data, tokenizer, label2id, max_length=512):
    inputs, all_labels = [], []

    for item in data:
        email_text = " ".join(email["body"].replace("\n", " ") for email in item["email_chain"])
        email_text_lower = email_text.lower()

        profile = item.get("tenant_profile", {})
        prefs = profile.get("Property Preferences", {})

        # Get all entities
        raw_entities = {
            "FirstName": profile.get("Tenant Representative Details", {}).get("First Name", ""),
            "LastName": profile.get("Tenant Representative Details", {}).get("Last Name", ""),
            "Email": profile.get("Tenant Representative Details", {}).get("Email", ""),
            "Phone": profile.get("Tenant Representative Details", {}).get("Phone", ""),
            "CompanyName": profile.get("Company Details", {}).get("Company Name", ""),
            "Industry": profile.get("Company Details", {}).get("Industry", ""),
            "CompanySize": profile.get("Company Details", {}).get("Company Size", ""),
            "GrowthStage": profile.get("Company Details", {}).get("Growth Stage", ""),
            "CurrentNeighborhood": profile.get("Company Details", {}).get("Current Neighborhood", []),
            "FirstInteraction": profile.get("First Interaction", ""),
            "LastInteraction": profile.get("Last Interaction", ""),
            "DecisionMakerRole": profile.get("Decision-Maker Role", ""),
            "PropertyType": prefs.get("Property Type", ""),
            "PreferredNeighborhood": prefs.get("Preferred Neighborhood", []),
            "Budget": prefs.get("Estimated or Stated Budget", ""),
            "MustHaves": prefs.get("Must-Haves", []),
            "NiceToHaves": prefs.get("Nice-to-Haves", []),
            "SpaceSize": prefs.get("Space Size", ""),
            "PreferredLeaseTerm": prefs.get("Preferred Lease Term", ""),
            "MovingTerm": prefs.get("Moving Term", ""),
            "min_months": prefs.get("Moving Timing", {}).get("min_months", ""),
            "max_months": prefs.get("Moving Timing", {}).get("max_months", ""),
            "MovingTimeline": profile.get("Moving Timeline", ""),
            "PainPoint": profile.get("Pain Points", []),
            "UrgencyScore": str(profile.get("Urgency Score", "")),
            "Outcome": profile.get("Outcome", ""),
            "Personality": profile.get("Tenant Personality", "")
        }

        # Normalize all values to a list of values
        entities = {}
        for key, val in raw_entities.items():
            if isinstance(val, list):
                entities[key] = [str(v).strip() for v in val if v]
            elif val:
                entities[key] = [str(val).strip()]
            else:
                entities[key] = []

        # Tokenize input
        encoding = tokenizer(
            email_text,
            truncation=True,
            padding="max_length",
            max_length=max_length,
            return_offsets_mapping=True
        )
        offset_mapping = encoding.pop("offset_mapping")
        input_ids = encoding["input_ids"]

        # Set default label to "O"
        label_list = ["O"] * len(input_ids)

        # Tag tokens with BIO
        for entity_type, values in entities.items():
            for value in values:
                start = email_text_lower.find(value.lower())
                if start == -1:
                    continue
                end = start + len(value)

                inside = False
                for i, (token_start, token_end) in enumerate(offset_mapping):
                    if token_start == 0 and token_end == 0:
                        continue
                    if token_start >= end or token_end <= start:
                        continue
                    tag = f"I-{entity_type}" if inside else f"B-{entity_type}"
                    label_list[i] = tag
                    inside = True

        # Convert tags to IDs and mask padding
        label_ids = [label2id.get(tag, label2id["O"]) for tag in label_list]
        label_ids = [
            label_id if input_id != tokenizer.pad_token_id else -100
            for label_id, input_id in zip(label_ids, input_ids)
        ]

        inputs.append(encoding)
        all_labels.append(label_ids)

    return inputs, all_labels


In [ ]:
"""
    Creates a PyTorch Dataset from the pre-tokenized input encodings and their corresponding label sequences.

    Args:
        encodings (list of dict): List of dictionaries containing tokenized inputs.
        labels (list of list): List of label ID sequences aligned with the tokenized inputs.

    Methods:
        __getitem__(index): Returns a single item from the dataset as a dictionary
                          with input tensors and corresponding label tensor.
        __len__(): Returns the number of items in the dataset.
"""
class EmailNERDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, index):
        item = {
            key: torch.tensor(val)
            for key, val in self.encodings[index].items()
        }
        item["labels"] = torch.tensor(self.labels[index])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
'''
Define tokenizer and import pre-trained model
'''
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
'''
Select ENTITY_TAGS for BIO tagging, having "O" as the default tag
'''

ENTITY_TAGS = [
    "O",
    "B-FirstName", "I-FirstName",
    "B-LastName", "I-LastName",
    "B-Email", "I-Email",
    "B-Phone", "I-Phone",
    "B-CompanyName", "I-CompanyName",
    "B-Industry", "I-Industry",
    "B-CompanySize", "I-CompanySize",
    "B-GrowthStage", "I-GrowthStage",
    "B-CurrentNeighborhood", "I-CurrentNeighborhood",
    "B-FirstInteraction", "I-FirstInteraction",
    "B-LastInteraction", "I-LastInteraction",
    "B-DecisionMakerRole", "I-DecisionMakerRole",
    "B-PropertyType", "I-PropertyType",
    "B-PreferredNeighborhood", "I-PreferredNeighborhood",
    "B-Budget", "I-Budget",
    "B-SpaceSize", "I-SpaceSize",
    "B-PreferredLeaseTerm", "I-PreferredLeaseTerm",
    "B-MovingTerm", "I-MovingTerm",
    "B-min_months", "I-min_months",
    "B-max_months", "I-max_months",
    "B-MovingTimeline", "I-MovingTimeline",
    "B-MustHaves", "I-MustHaves",
    "B-NiceToHaves", "I-NiceToHaves",
    "B-PainPoint", "I-PainPoint",
    "B-UrgencyScore", "I-UrgencyScore",
    "B-Outcome", "I-Outcome",
    "B-Personality", "I-Personality"
]

# Define label to id mappings
label2id = {label: i for i, label in enumerate(ENTITY_TAGS)}
id2label = {i: label for label, i in label2id.items()}

# Set inicial label weights
label_weights = torch.tensor([0.05] + [1.0] * (len(ENTITY_TAGS) - 1), dtype=torch.float)

In [ ]:
'''
Initialize model
'''

model = DistilBertForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(ENTITY_TAGS),
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
'''
Import data and preprocess it
'''

with open("train.json") as f:  # Change path according to the file location
    train_data = json.load(f)
with open("val.json") as f:  # Change path according to the file location
    val_data = json.load(f)
with open("test.json") as f:  # Change path according to the file location
    test_data = json.load(f)

'''
For running with github repo.:
with open("../3. Data Split/train.json") as f:
    train_data = json.load(f)
with open("../3. Data Split/val.json") as f:
    val_data = json.load(f)
with open("../3. Data Split/test.json") as f:
    test_data = json.load(f)
'''

# Tokenize
train_encodings, train_labels = preprocess_for_token_classification(train_data, tokenizer, label2id)
val_encodings, val_labels = preprocess_for_token_classification(val_data, tokenizer, label2id)
test_encodings, test_labels = preprocess_for_token_classification(test_data, tokenizer, label2id)

# Create Datasets
train_data = EmailNERDataset(train_encodings, train_labels)
val_data = EmailNERDataset(val_encodings, val_labels)
test_data = EmailNERDataset(test_encodings, test_labels)


In [ ]:
# Check tokenized outputs
for name, dataset in zip(["Validation", "Test", "Train"], [val_data, test_data, train_data]):
  print(f"\nLabel distribution for {name} dataset:")
  all_labels = sum([item['labels'].tolist() for item in dataset], [])
  label_counts = Counter(all_labels)
  print({(id2label[k] if k != -100 else 'IGNORED'): v for k, v in label_counts.items()})


Label distribution for Validation dataset:
{'O': 12598, 'B-FirstName': 30, 'B-CompanyName': 25, 'I-CompanyName': 37, 'B-MustHaves': 60, 'I-MustHaves': 60, 'B-UrgencyScore': 21, 'IGNORED': 2406, 'B-LastName': 12, 'B-Industry': 13, 'B-CurrentNeighborhood': 2, 'I-CurrentNeighborhood': 2, 'B-PropertyType': 27, 'B-PreferredNeighborhood': 9, 'I-PreferredNeighborhood': 8, 'B-NiceToHaves': 10, 'I-NiceToHaves': 13, 'B-PreferredLeaseTerm': 4, 'I-PreferredLeaseTerm': 5, 'B-Personality': 2, 'B-DecisionMakerRole': 7, 'I-FirstName': 2, 'B-Outcome': 2, 'B-SpaceSize': 1, 'I-SpaceSize': 4}

Label distribution for Test dataset:
{'O': 12209, 'B-FirstName': 30, 'B-CompanyName': 26, 'I-CompanyName': 38, 'B-PreferredNeighborhood': 13, 'I-PreferredNeighborhood': 12, 'B-PropertyType': 26, 'B-MustHaves': 49, 'I-MustHaves': 60, 'B-UrgencyScore': 20, 'IGNORED': 2818, 'B-PreferredLeaseTerm': 3, 'I-PreferredLeaseTerm': 3, 'B-LastName': 8, 'B-NiceToHaves': 10, 'I-NiceToHaves': 10, 'B-Industry': 12, 'B-DecisionMake

In [ ]:
"""
  A subclass of DistilBertForTokenClassification that incorporates class weights into the loss function.

  This model is designed for token classification tasks where class imbalance may exist.
  It uses a cross-entropy loss function with class weights and `ignore_index=-100`
  to ignore padding tokens.

  Args:
      - config: Model configuration.
      - class_weights (torch.Tensor, optional): 1D tensor of weights for each class. If None, standard (unweighted) loss is used.
"""

class WeightedDistilBertForTokenClassification(DistilBertForTokenClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        self.class_weights = class_weights

    def forward(self, input_ids=None, attention_mask=None, num_items_in_batch=None, labels=None, **kwargs):
        '''
          Forward Args:
              - input_ids (torch.LongTensor): Token IDs with shape (batch_size, sequence_length).
              - attention_mask (torch.LongTensor): Mask to avoid performing attention on padding tokens.
              - labels (torch.LongTensor, optional): Token-level labels for computing the loss.
              - num_items_in_batch (optional): Unused placeholder for potential logging/debugging.
              - **kwargs: Additional keyword arguments for base model.

          Returns:
              - dict: A dictionary with:
                  - "loss" (torch.FloatTensor, optional): The weighted cross-entropy loss, if labels are provided.
                  - "logits" (torch.FloatTensor): The predicted logits of shape (batch_size, sequence_length, num_labels).
        '''
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=None, **kwargs)
        logits = outputs.logits
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(logits.dtype).to(logits.device), ignore_index=-100)
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}


In [ ]:
'''
Initialize model
'''
# Load configuration with entities, labels and label ids
config = DistilBertConfig.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(ENTITY_TAGS),
    id2label=id2label,
    label2id=label2id
)

# Load model using weighted class
model = WeightedDistilBertForTokenClassification.from_pretrained(
    "distilbert-base-uncased",
    config=config,
    class_weights=label_weights
)

Some weights of WeightedDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
'''
Define metrics for model evaluation
'''

def compute_metrics(p):
    predictions, labels = p
    preds = predictions.argmax(axis=-1)

    true_labels = []
    pred_labels = []

    for pred, label in zip(preds, labels):
        true_seq = []
        pred_seq = []
        for p_, l_ in zip(pred, label):
            if l_ != -100: # Ignore padding
                true_seq.append(id2label[l_])
                pred_seq.append(id2label[p_])
        true_labels.append(true_seq)
        pred_labels.append(pred_seq)

    return {
        "precision": precision_score(true_labels, pred_labels, zero_division=1),
        "recall": recall_score(true_labels, pred_labels, zero_division=1),
        "f1": f1_score(true_labels, pred_labels, zero_division=1)
    }


In [ ]:
"""
Extract contiguous entities from BIO-tagged tokens.

Args:
    tokens (List[str]): List of tokens.
    labels (List[str]): Corresponding BIO labels (e.g., B-Budget, I-Budget, O).

Returns:
    Dict[str, str]: A mapping from field name to extracted string value.
"""
def extract_entities_from_tokens(tokens, labels):

    entity_dict = {}
    current_entity = None
    current_tokens = []

    for token, label in zip(tokens, labels):
        if label == "O" or token in tokenizer.all_special_tokens:
            if current_entity:
                value = tokenizer.convert_tokens_to_string(current_tokens).replace(" ##", "")
                entity_dict.setdefault(current_entity.lower(), value.strip())
                current_entity, current_tokens = None, []
            continue

        tag, field = label.split("-", 1)

        if tag == "B":
            if current_entity:
                value = tokenizer.convert_tokens_to_string(current_tokens).replace(" ##", "")
                entity_dict.setdefault(current_entity.lower(), value.strip())
            current_entity = field
            current_tokens = [token]
        elif tag == "I" and field == current_entity:
            current_tokens.append(token)
        else:
            if current_entity:
                value = tokenizer.convert_tokens_to_string(current_tokens).replace(" ##", "")
                entity_dict.setdefault(current_entity.lower(), value.strip())
            current_entity = None
            current_tokens = []

    if current_entity and current_tokens:
        value = tokenizer.convert_tokens_to_string(current_tokens).replace(" ##", "")
        entity_dict.setdefault(current_entity.lower(), value.strip())

    return entity_dict


In [ ]:
"""
Convert BERT predictions on token-level to structured tenant profiles.

Args:
    dataset: EmailNERDataset instance
    predictions: raw logits from trainer.predict()
    id2label: mapping from label id to BIO label
    tokenizer: tokenizer used for encoding

Returns:
    List[Dict]: structured tenant profiles
"""
def convert_predictions_to_profiles(dataset, predictions, id2label, tokenizer):

    predicted_ids = predictions.argmax(axis=-1)
    profiles = []

    for i in range(len(dataset)):
        input_ids = dataset[i]["input_ids"]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        label_ids = predicted_ids[i]


        tokens_filtered = []
        labels_filtered = []
        for token, label_id in zip(tokens, label_ids):
            if label_id == -100 or token == tokenizer.pad_token: # Ignore padding
                continue
            tokens_filtered.append(token)
            labels_filtered.append(id2label[label_id])

        entities = extract_entities_from_tokens(tokens_filtered, labels_filtered)

        # Fill with all expected keys
        structured_profile = {
            key.lower(): entities.get(key.lower(), None)
            for key in [
                "FirstName", "LastName", "Email", "Phone",
                "CompanyName", "Industry", "CompanySize", "GrowthStage",
                "CurrentNeighborhood", "FirstInteraction", "LastInteraction",
                "DecisionMakerRole", "PropertyType", "PreferredNeighborhood",
                "Budget", "SpaceSize", "PreferredLeaseTerm", "MovingTerm",
                "min_months", "max_months", "MovingTimeline",
                "MustHaves", "NiceToHaves", "PainPoint",
                "UrgencyScore", "Outcome", "Personality"
            ]
        }
        profiles.append(structured_profile)

    return profiles


In [ ]:
"""
  Extracts f1, Precsion and Recall for each entity (column) on the dataframe.

  Args:
      - df (pandas Dataframe):

"""

def get_entity_metrics(df):
    field_names = df.columns.tolist()
    results = {}
    macro_p, macro_r, macro_f1 = [], [], []

    for field in field_names:
        y_true = df[field].fillna("").str.strip().str.lower().tolist()
        y_pred = df[field].fillna("").str.strip().str.lower().tolist()

        binary_true = [1 if val else 0 for val in y_true]
        binary_pred = [1 if pred == val and val else 0 for pred, val in zip(y_pred, y_true)]

        if any(binary_true):
            p = sk_precision_score(binary_true, binary_pred, zero_division=0)
            r = sk_recall_score(binary_true, binary_pred, zero_division=0)
            f1 = sk_f1_score(binary_true, binary_pred, zero_division=0)
            results[f"{field}_precision"] = round(p, 4)
            results[f"{field}_recall"] = round(r, 4)
            results[f"{field}_f1"] = round(f1, 4)
            macro_p.append(p)
            macro_r.append(r)
            macro_f1.append(f1)

    results["macro_precision"] = round(np.mean(macro_p), 4) if macro_p else 0.0
    results["macro_recall"] = round(np.mean(macro_r), 4) if macro_r else 0.0
    results["macro_f1"] = round(np.mean(macro_f1), 4) if macro_f1 else 0.0


    print("Structured Field-Level Metrics:", results)
    return results


In [ ]:
"""
  Evaluates a token classification model on the given dataset and prints a classification report.

  This function assumes that the dataframe contains one column per entity field, with
  predictions and ground truths already aligned. It treats each cell as correct if
  the predicted value matches the true value exactly (case-insensitive and stripped).

  Args:
      - df (pandas.DataFrame): DataFrame containing one column per entity. Each column
                               should contain both ground truth and predicted values
                               already aligned row-wise.

  Returns:
     - dict: A dictionary with precision, recall, and F1 score for each field, as well
             as macro-averaged precision, recall, and F1 score across all fields.

"""
def evaluate_model(trainer, dataset, id2label, tokenizer):
    predictions, labels, _ = trainer.predict(dataset)
    preds = predictions.argmax(axis=-1)

    true_labels = []
    pred_labels = []

    for pred, label in zip(preds, labels):
        true_seq = []
        pred_seq = []
        for p, l in zip(pred, label):
            if l != -100:
                true_seq.append(id2label[l])
                pred_seq.append(id2label[p])
        true_labels.append(true_seq)
        pred_labels.append(pred_seq)

    profiles = convert_predictions_to_profiles(test_data, predictions, id2label, tokenizer)
    df_profiles = pd.DataFrame(profiles)

    print(seqeval_classification_report(true_labels, pred_labels))

    return df_profiles


In [ ]:
'''
Initialize wandb
'''

wandb.init()

wandb: Currently logged in as: lidianeh (lidianeh-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
'''
Define sweep configurations and set up sweep agent
'''

sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "eval_f1",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "distribution": "uniform",
            "min": 1e-5,
            "max": 4e-5
        },
        "per_device_train_batch_size": {
            "values": [8, 16]
        },
        "num_train_epochs": {
            "values": [6, 8, 10]
        },
        "weight_decay": {
            "distribution": "uniform",
            "min": 0.05,
            "max": 0.2
        },
        "warmup_ratio": {
            "distribution": "uniform",
            "min": 0.05,
            "max": 0.2
        },
        "adam_epsilon": {
            "values": [1e-8, 1e-6]
        },
        "seed": {
            "values": [42, 2024]
        }
    }
}


sweep_id = wandb.sweep(sweep_config, project="distilbert-final")

Create sweep with ID: 6yae4880
Sweep URL: https://wandb.ai/lidianeh-none/distilbert-final/sweeps/6yae4880


In [ ]:
"""
Trains and evaluates DistilBERT-based token classification model using a W&B sweep configuration.

This function initializes wandb, builds a model and training pipeline using hyperparameters from sweep_config,
training and evaluating the model on the provided datasets, and logging evaluation metrics to W&B.

Args:
    - config (dict, optional): Sweep configuration containing training hyperparameters.
      If None, wandb will use the default sweep config.

Raises:
    - Exception: Prints the error and finishes the wandb run if training fails.
"""


def train_with_wandb_sweep(config=None):

    with wandb.init(config=config) as run:
        config = wandb.config

        model_config = DistilBertConfig.from_pretrained(
            "distilbert-base-uncased",
            num_labels=len(ENTITY_TAGS),
            id2label=id2label,
            label2id=label2id
        )

        model = WeightedDistilBertForTokenClassification.from_pretrained(
            "distilbert-base-uncased",
            config=model_config,
            class_weights=label_weights
        )

        training_args = TrainingArguments(
            output_dir="./results",
            num_train_epochs=config.num_train_epochs,
            per_device_train_batch_size=config.per_device_train_batch_size,
            per_device_eval_batch_size=config.per_device_train_batch_size,
            learning_rate=config.learning_rate,
            weight_decay=config.weight_decay,
            warmup_ratio=config.warmup_ratio,
            adam_epsilon=config.adam_epsilon,
            seed=config.seed,
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_dir="./logs",
            logging_strategy="epoch",
            logging_steps=10,
            report_to="wandb",
            load_best_model_at_end=True,
            metric_for_best_model="eval_f1",
            greater_is_better=True
        )


        # Initialize trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_data,
            eval_dataset=val_data,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # Train and evaluate
        try:
            trainer.train()
            trainer.evaluate()
            metrics = trainer.evaluate(test_data)
            print(metrics)
            df = evaluate_model(trainer, test_data, id2label, tokenizer)
            print(df)
            wandb_table = wandb.Table(dataframe=df)
            wandb.log({f"{run.name} Predictions": wandb_table})

            entity_metrics = get_entity_metrics(df)
            wandb.log({**{f"test_{k}": float(v) for k, v in entity_metrics.items()}})

            # Save all predictions to Google Drive
            pred_path = os.path.join(output_dir, f"{wandb.run.name}_predictions.json")
            df.to_json(pred_path, orient="records", indent=2)
            print(f"Predictions saved to: {pred_path}")

        except Exception as e:
            print(f"Training failed: {e}")
            return


In [ ]:
''' Launch the sweep agent '''
wandb.agent(sweep_id, function=train_with_wandb_sweep, count=25)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: o0qra5bp with config:
wandb: 	adam_epsilon: 1e-08
wandb: 	learning_rate: 2.652952326169707e-05
wandb: 	num_train_epochs: 8
wandb: 	per_device_train_batch_size: 8
wandb: 	seed: 2024
wandb: 	warmup_ratio: 0.07452962363800385
wandb: 	weight_decay: 0.11292689409200424


Some weights of WeightedDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-19-362b024c7306>:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Confi

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,3.056600,1.847368,1.000000,0.000000,0.000000
2,1.482800,1.168944,0.301075,0.246696,0.271186
3,0.979900,0.812521,0.244660,0.555066,0.339623
4,0.726400,0.646151,0.237013,0.643172,0.346382
5,0.581400,0.566431,0.250000,0.682819,0.365998
6,0.489200,0.520067,0.229137,0.713656,0.346895
7,0.447800,0.498501,0.269737,0.722467,0.392814
8,0.415000,0.493678,0.273345,0.709251,0.394608


{'eval_loss': 0.42069491744041443, 'eval_precision': 0.30326295585412666, 'eval_recall': 0.7632850241545893, 'eval_f1': 0.4340659340659341, 'eval_runtime': 0.2116, 'eval_samples_per_second': 141.81, 'eval_steps_per_second': 18.908, 'epoch': 8.0}


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                       precision    recall  f1-score   support

          CompanyName       0.38      0.67      0.48        27
  CurrentNeighborhood       0.00      0.00      0.00         1
    DecisionMakerRole       0.00      0.00      0.00         6
            FirstName       0.37      0.90      0.52        30
             Industry       0.00      0.00      0.00        12
             LastName       0.00      0.00      0.00         8
            MustHaves       0.20      0.56      0.30        50
          NiceToHaves       0.00      0.00      0.00        10
            PainPoint       0.00      0.00      0.00         1
   PreferredLeaseTerm       0.00      0.00      0.00         3
PreferredNeighborhood       0.00      0.00      0.00        13
         PropertyType       1.00      0.38      0.56        26
         UrgencyScore       0.00      0.00      0.00        20

            micro avg       0.31      0.40      0.35       207
            macro avg       0.15      0.19      0.14 

eval/f1,▁▁▁▂▆▇▇█
eval/loss,█▄▃▂▁▁▁▁
eval/precision,███▁▂▂▂▂
eval/recall,▁▁▁▂▆▇▇█
eval/runtime,▇▁▂▁▄██▆
eval/samples_per_second,▂█▇█▅▁▁▃
eval/steps_per_second,▂█▇█▅▁▁▃
test/f1,▁
test/loss,▁
test/precision,▁
test/recall,▁


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7cf499a5bf10>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
'''
Predict and Evaluate results on DistilBERT model without training
'''

def run_base_model():
    print("Running base DistilBERT model without training...")

    with wandb.init(project="distilbert-final", name="distilbert_base", reinit=True) as run:

        # Load pre-trained base model
        base_model = DistilBertForTokenClassification.from_pretrained(
            "distilbert-base-uncased",
            num_labels=len(ENTITY_TAGS),
            id2label=id2label,
            label2id=label2id
        ).to("cuda" if torch.cuda.is_available() else "cpu")

        # Setup trainer
        eval_args = TrainingArguments(
            output_dir="./base_model_results",
            per_device_eval_batch_size=8,
            report_to="wandb",
            logging_dir="./logs",
            seed=42
        )

        # Initiate trainer
        trainer = Trainer(
            model=base_model,
            args=eval_args,
            eval_dataset=test_data,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # Evaluate trainer
        trainer.evaluate()
        metrics = trainer.evaluate(test_data)
        print(metrics)
        df = evaluate_model(trainer, test_data, id2label, tokenizer)
        print(df)
        wandb_table = wandb.Table(dataframe=df)
        wandb.log({f"{run.name} Predictions": wandb_table})

        entity_metrics = get_entity_metrics(df)
        wandb.log({**{f"test_{k}": float(v) for k, v in entity_metrics.items()}})

        # Save all predictions to Google Drive
        pred_path = os.path.join(output_dir, f"{wandb.run.name}_predictions.json")
        df.to_json(pred_path, orient="records", indent=2)
        print(f"Predictions saved to: {pred_path}")

#run_base_model()